In [17]:
liked_books = [4408, 31147619, 29983711, 9401317, 9317691, 8153988]

In [18]:
book_mapping = {}

with open("book_id_map.csv", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        csv_id, book_id = line.split(",")
        try:
            book_id = int(book_id)
        except ValueError:
            continue
        book_mapping[book_id] = csv_id

In [19]:
csv_ids = [book_mapping[item] for item in liked_books]
csv_ids

['50894', '69614', '16177', '704382', '69995', '112917']

In [4]:
overlap_users = set()

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, book_id, _, rating, _ = line.split(",")

        try:
            rating = int(rating)
        except ValueError:
            continue
        
        if book_id in csv_ids and rating >= 4:
            overlap_users.add(user_id)

In [5]:
len(overlap_users)

1974

In [6]:
rec_lines = []

with open("goodreads_interactions.csv", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        user_id, book_id, _, rating, _ = line.split(",")
        
        if user_id in overlap_users:
            rec_lines.append([user_id, book_id, rating])

In [7]:
import pandas as pd

recs = pd.DataFrame(rec_lines, columns=["user_id", "book_id", "rating"])

In [8]:
recs.head()

,user_id,book_id,rating
0,284,68224,3
1,284,68223,4
2,284,68222,3
3,284,68221,4
4,284,68220,5


In [9]:
recs

,user_id,book_id,rating
0,284,68224,3
1,284,68223,4
2,284,68222,3
3,284,68221,4
4,284,68220,5
...,...,...,...
1504903,873216,13213,0
1504904,873216,8400,0
1504905,873216,7116,0
1504906,873216,416993,0


In [51]:
top_recs = recs["book_id"].value_counts().head(25)

In [52]:
top_recs = top_recs.index.values

In [53]:
top_rec_ids = [k for k,v in book_mapping.items() if v in top_recs]

In [21]:
import json

with open("books_titles.json", "r") as f:
    books_titles = json.load(f)

In [55]:
books_titles = pd.DataFrame.from_dict(books_titles)
books_titles["book_id"] = pd.to_numeric(books_titles["book_id"])

In [24]:
books_titles.head()

,book_id,title,ratings,url
0,1333909,Good Harbor,10,https://www.goodreads.com/book/show/1333909.Go...
1,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...
2,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...
3,287140,Runic Astrology: Starcraft and Timekeeping in ...,15,https://www.goodreads.com/book/show/287140.Run...
4,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...


In [56]:
books_titles[books_titles["book_id"].isin(top_rec_ids)]

,book_id,title,ratings,url
71398,77203,The Kite Runner,1848782,https://www.goodreads.com/book/show/77203.The_...
99361,7260188,"Mockingjay (The Hunger Games, #3)",1743362,https://www.goodreads.com/book/show/7260188-mo...
386663,2767052,"The Hunger Games (The Hunger Games, #1)",4899965,https://www.goodreads.com/book/show/2767052-th...
507835,19063,The Book Thief,1193697,https://www.goodreads.com/book/show/19063.The_...
540144,7624,Lord of the Flies,1638289,https://www.goodreads.com/book/show/7624.Lord_...
546297,5107,The Catcher in the Rye,2086945,https://www.goodreads.com/book/show/5107.The_C...
608482,5907,The Hobbit,2099680,https://www.goodreads.com/book/show/5907.The_H...
630937,4671,The Great Gatsby,2758812,https://www.goodreads.com/book/show/4671.The_G...
649821,1,Harry Potter and the Half-Blood Prince (Harry ...,1713866,https://www.goodreads.com/book/show/1.Harry_Po...
711064,4667024,The Help,1565302,https://www.goodreads.com/book/show/4667024-th...
